# preparing metadata

In this notebook, I describe how we prepare the sample set of annotated advertisements. <br>
Using the annotations, we count the number of males and females in each advertisements

In [ ]:
import sys
import seaborn as sns
import pickle
from tqdm import tqdm
from scipy.spatial import distance as dist
# set system path for importing modules
sys.path.append('../src/')
output_dir = '../reports/figures/'

In [4]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import xml.etree.ElementTree as ET

from utils import find_bb, to_parseable, get_men_women_annotations

In [5]:
%matplotlib inline

Using 'make_samples.py' we draw a 1000 samples per year from the SIAMESET

First we load the annotations

In [6]:
annotations_path = '../data/raw/annotations/'
annotations = glob.glob('../data/raw/annotations/*.xml')
annotations_base = [os.path.splitext(os.path.basename(annotation))[0] for annotation in annotations]

Second, we load the metadata belonging to the sampled ads. <br>
The metadata file refers to all the ads in the SiameSet. <br>
We filter based on ads that are also in the annotation set.

In [7]:
ad_meta = pd.read_csv('../data/raw/ads_meta.csv')
ad_meta['date'] = pd.to_datetime(ad_meta['date'])
ad_meta = ad_meta[ad_meta['id'].str.replace(':', '-').isin(annotations_base)]

print('total number of ads with an annotation: {}'.format(ad_meta.shape[0]))

total number of ads with an annotation: 8522


In [17]:
## TODO: REFACTOR THIS!!

men_count = []
ad_with_people_count = 0
women_count = []


areas_m_list = []
areas_f_list = []
distances_m_list = []
distances_f_list = []
total_areas = []

men = {}
women = {}

for index, row in tqdm(ad_meta.iterrows(), total=ad_meta.shape[0]):
    file_base = row['id'].replace(':', '-')
    xml = annotations_path + file_base + '.xml'
    men, women, areas_m, areas_f, distances_m, distances_f, position_m, position_f, object_m, object_f, total_area = get_men_women_annotations(xml, prediction=False)
    total_areas.append(total_area)
    if men > 0 or women > 0:
        ad_with_people_count += 1
        men_count.append(men)
        women_count.append(women)
        areas_m_list.append(areas_m)
        areas_f_list.append(areas_f)
        distances_m_list.append(distances_m)
        distances_f_list.append(distances_f)
    else:
        men_count.append(0)
        women_count.append(0)
        areas_m_list.append(0)
        areas_f_list.append(0)
        distances_m_list.append(0)
        distances_f_list.append(0)
     




# print('total number of men: {}'.format(ad_meta['men_count'].sum()))
# print('total number of women: {}'.format(ad_meta['women_count'].sum()))

100%|██████████| 8150/8150 [34:02<00:00,  3.99it/s]  


ValueError: Length of values does not match length of index

In [24]:
ad_meta['men_count'] = men_count
ad_meta['women_count'] = women_count
ad_meta['size'] = total_areas
ad_meta['men_areas'] = areas_m_list
ad_meta['women_areas'] = areas_f_list

In [25]:
ad_meta['total_count'] = ad_meta['men_count'] + ad_meta['women_count']
ad_meta['year'] = pd.to_datetime(ad_meta['date']).dt.year
#ad_meta = ad_meta[ad_meta['total_count'] != 0]


In [27]:
ad_meta.to_csv('../data/processed/counts.csv', index=0)